# ASTR 695 Python Seminar
> ### Presented by Ramsey Karim

Contents:<br>
[Section 1: Code Headers](#sec_headers)<br>
[Section 2: numpy and scipy](#sec_np)<br>
[Section 3: Advanced Plotting](#sec_plotting)<br>
[Section 4: astropy](#sec_astropy)<br>
[Section 5: Classes and Object-oriented Programming](#sec_classes)<br>
[Section 6: Python built-ins](#sec_builtins)<br>

***
<a id='sec_headers'></a>
## 1. Code headers
#### Document your code!!!11!

I don't usually code in Jupyter Notebooks, so bear with me as I figure out the commands! I usually code in indivudual `.py` files, sometimes strung together in modules. Whether part of a module or a standalone file, I like to start each file with a sort of standardized header including information about what the script is for, when I created it, when I last updated it, and (just in case someone finds it on Github) some contact info.

As I'm sure some of you know, you will write excellent, useful code this month, next month, next February. Two or three years later, you will work on a similar problem. You will think to yourself, "Hey, I have done this before". You will look for your old code. You will wonder which file that function is in. You will wonder what the function was for, and if/when you modified it.
Or, you will return to a project after 6 months and try to quickly pick up where you left off. You will appreciate the date stamps and header text explaining what each file was for.

In [ ]:
"""
Notebook for the Python seminar, ASTR 695, Fall 2020
Created: 2020-09-08
Updated: 2020-09-13
Presented: 2020-09-14
"""
__author__ = "Ramsey Karim"
__email__ = "rlkarim@umd.edu"

What will your standard header look like? Check out __[this StackOverflow question](https://stackoverflow.com/questions/1523427/what-is-the-common-header-format-of-python-files)__ for more info and ideas, or search for or invent a header format that works best for you!

***
***
<a id='sec_np'></a>
## 2. `numpy` and `scipy`

### 2.1 `numpy`
The `numpy` package implements array data types and functions for those array types.

Python offers the `list` class as a built-in data type. Python lists are the most "pythonic" way to store sequences of objects. The objects do not have to be the same type. You can build  a list of lists to represent a 2D array, and nest for-loops to loop through it. You can change the elements of a list, including adding and removing elements anywhere in the list, without creating a new list.

Numpy offers the `numpy.ndarray` class, which creates fixed-size arrays of user-specified dimension, and restricts the array element data type to be the same all throughout the array. You cannot resize the array without making a new one, but you can change the values of array elements.

The fixed-size and fixed-type restrictions allow `numpy` functions to work on these arrays _much faster_ than you can process Python `list` items in a for-loop. If you are operating on a set of numbers, I _strongly recommend_ using `numpy` instead of lists.

The Python `list` is still very useful! In any case except the above, the `list` may be more efficient and readable. If you need to append/insert/remove elements to/into/from the sequence, lists are great! If your sequence elements are not numbers, but something like strings, user-defined classes, etc, lists are great!

In [ ]:
import numpy as np

***
#### 2.1.1 When is `numpy` faster?

In [ ]:
# Change the list length (try 100, 10000, 1000000) and rerun the following cells to see how runtime changes.
list_length = 100

In [ ]:
# Use numpy to create an array
arr = np.arange(list_length, dtype=np.float64)

In [ ]:
%%time
arr += 3
arr[arr > 40] *= 5.

In [ ]:
# Create a similar Python list
l = list(float(x) for x in range(list_length))

In [ ]:
%%time
for i, element in enumerate(l):
    element += 3
    if element > 40:
        element *= 5
    l[i] = element

Python `list` is faster for small list lengths. However, when you get up to list lengths of $\sim 10^5$ elements, our simple operation runs around an order of magnitude faster with `numpy.ndarray`.

***
#### `numpy` functions
Numpy has tons of mathematical functions available. If you can use them correctly, they'll be much faster than looping over a Python list. Google around for "numpy function_I_want" when you're stuck. If Numpy doesn't have it, Scipy might!

When you are using Numpy arrays, use Numpy functions whenever possible! They're fast and they're readable. Don't manually loop over your `numpy.ndarray` if you can avoid it!

In [ ]:
l_as_array = np.array(l)
print(f"Array of shape: {l_as_array.shape}, size: {l_as_array.size}")

print(f"Mean: {np.mean(l_as_array):.3f}")
print(f"Median: {np.median(l_as_array):.3f}")

***
#### 2.1.2 When should I use a Python `list`?
Like I mentioned before, there are times to use a list instead. If your primary operations are appending to a sequence, or removing from a sequence, or something else that won't work well with a pre-sized array, use a `list`. For example, building up a sequence of unknown final length:

In [ ]:
# Who knows when our sequence will end!
# Change the stop condition and rerun the following cells. Try 1e2 and 1e5
unknown_stopping_condition = lambda x: x > 1e2

In [ ]:
%%time
# Array version
i = 0
arr = np.array([])
while not unknown_stopping_condition(i):
    arr = np.append(arr, [i])
    i += 1
print(f"Final array size: {arr.size}")

In [ ]:
%%time
# List version
i = 0
l = []
while not unknown_stopping_condition(i):
    l.append(i)
    i += 1
l_as_array = np.array(l)
print(f"Final array size: {l_as_array.size}")

By the time we get to $10^5$ elements, the `numpy.ndarray` method takes several seconds, but the list method is still milliseconds. Numpy functions that resize the array often have to build a new array and return that. So instead of reusing the same array in memory most of the time like `list` does, Numpy builds a brand new array every single iteration.

#### Into the Weeds:
You might have correctly guessed that Numpy arrays are implemented as C arrays under the hood, since they're fixed-size and fixed-type. Python lists are also implemented using C arrays, [but they behave differently](https://stackoverflow.com/questions/3917574/how-is-pythons-list-implemented)! Learn how your data types are implemented in order to use them most efficiently.

***
### 2.2 `scipy`
The `scipy`package implements a variety of functions that are useful in scientific applications.

I frequently use the `scipy.optimize` library, especially the `curve_fit` and `minimize` functions. I also really like the `scipy.constants` library of fundamental constants (`astropy.constants` offers a similar library. I think you can just pick one to get familiar with).

***
#### 2.2.1 curve_fit example

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# Use curve_fit to fit a Gaussian distribution
from scipy.optimize import curve_fit

# Get random number generator from numpy.random
# Numpy told me to do this (https://numpy.org/devdocs/reference/random/index.html#random-quick-start)
rng = np.random.default_rng()

# Draw a bunch of samples from a Gaussian distribution
samples = rng.normal(loc=0., scale=2., size=10000)
hist_samples, bin_edges = np.histogram(samples, bins=32)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Make nice, plottable histogram arrays
hist_x = np.concatenate(tuple(zip(bin_edges[:-1], bin_edges[1:])))
hist_y = np.concatenate(tuple(zip(hist_samples, hist_samples)))
plt.plot(hist_x, hist_y, label='Data')

# Define your Gaussian function to fit
def gaussian(x, mu, sigma, A):
    """
    Nice, simple Gaussian function to give to curve_fit
    The first argument must be an array of input values
    The rest of the arguments will be fit by curve_fit
    :param x: array of input values
    :param mu: mean of the distribution
    :param sigma: standard deviation of the distribution
    :param A: amplitude of the distribution
    :returns: array of the same shape as the input array x
    """
    return A * np.exp(-0.5 * (x - mu)**2. / sigma**2.) / (sigma * np.sqrt(2 * np.pi))

# Make some initial guesses for the parameters mu, sigma, and A
guesses = [1., 3.5, 500]

# Send it to curve_fit!
popt, pcov = curve_fit(gaussian, bin_centers, hist_samples, p0=guesses)
# pcov is the covariance matrix
errors = np.sqrt(np.diag(pcov))
print(errors)
print(f"True mean = 0, fitted: {popt[0]:.2f} +/- {errors[0]:.3f}")
print(f"True standard deviation = 2, fitted: {popt[1]:.2f} +/- {errors[1]:.3f}")
print(f"Fitted amplitude: {popt[2]:.1f} +/- {errors[2]:.1f}")
plt.plot(bin_centers, gaussian(bin_centers, *popt), '--', color='k', label='Fitted')
plt.legend();

***
#### 2.2.2 spline example

In [ ]:
# You could also try a spline interpolation of that same Gaussian with another scipy function
# Splines can be useful for modeling things like bandpass curves, where there's no functional definition
# There are 2D spline functions available in scipy as well!
from scipy.interpolate import UnivariateSpline

spline = UnivariateSpline(bin_centers, hist_samples, s=0)  # Look up this s (smoothing) parameter, it's useful!

plt.figure()
plt.title("Spline")
plt.plot(hist_x, hist_y, label='Data')
plt.plot(bin_centers, spline(bin_centers), '--', color='g', label='Spline')
plt.legend()

plt.figure()
plt.title("Spline vs Fit Residuals")
residuals = spline(bin_centers) - gaussian(bin_centers, *popt)
plt.plot(bin_centers, residuals, color='r')
plt.ylabel("Spline $-$ Fit");

There are plenty of other useful `scipy` functions. `minimize` is an alternative to `curve_fit` that doesn't need x or y array inputs; you give it a function that returns a single number, and then it varies the parameters until it finds the lowest value of that number. It's useful for fitting things that aren't one-dimensional "x versus y" problems. There are other interpolation functions in `scipy.interpolate`, which can be helpful while working with images or other data that don't have functional definitions. There are convolution routines in `scipy.signal`, Fourier transform routines in `scipy.fft` ([and](https://stackoverflow.com/questions/6363154/what-is-the-difference-between-numpy-fft-and-scipy-fftpack) `numpy.fft`), integration routines in `scipy.integrate`, linear algebra functions in `scipy.linalg`, stats things in `scipy.stats`. `scipy.spatial` has interesting things like Delaunay and Voronoi triangulation functions, and `scipy.ndimage` has some image processing functions.

You can find more `scipy` libraries and their documentation __[here](https://docs.scipy.org/doc/scipy/reference/)__!

### Most important takeaway:
If you ever find yourself writing up some function that seems pretty fundamental, like some reasonably well-known equation or algorithm, check `numpy`, `scipy`, and `astropy` first. Then Google it, see if it's in another package. Then check other places online, like StackExchange/StackOverflow or Github, to see if you can use/copy code. _Then_ think about writing it yourself.


The huge advantage of `numpy`, `scipy`, or `astropy` functions, or functions from other popular libraries, is that they've been tested and are used extensively within the community, so they're pretty trustworthy. They're more readable in your code too, and they're well documented online. Finding it elsewhere online is great for saving time, but remember that it might not have been tested as extensively.

***
***
<a id='sec_plotting'></a>
## 3. Advanced Plotting
Everything in this section uses `matplotlib` since that's what I'm most familiar with. I think all this stuff is correct.__*__

__*Big huge asterisk__, this is all stuff I learned from experience, so I'm sure if you gave all this notebook to a matplotlib software author, they'd make all kinds of corrections about some stuff I said not being technically correct. But I _can_ guarantee you that this stuff will work. Because I'll prove it. In this notebook.

In [ ]:
# In case you don't want to run all the previous cells
import numpy
import matplotlib.pyplot as plt

***
### 3.1 `Figure` and `Axes` objects
A `Figure` object loosely represents a single plot window, which may itself contain multiple plots, but is a coherent entity that would be saved to one single image file or takes up one single "window" in your [window manager](https://en.wikipedia.org/wiki/Window_manager). You can simply create these objects with `fig = plt.figure()`. More info on the `Figure` class [here](https://matplotlib.org/3.3.1/api/_as_gen/matplotlib.figure.Figure.html). I don't know what the specific meaning of figures is in jupyter notebooks, and the "window" definition seems to be lost in this format.

An `Axes` object loosely represents a single plot lying within a `Figure`. A `Figure` may contain more than one `Axes` objects (plots). There is also an `Axis` object in matplotlib, not to be confused with `Axes`, which represents a literal x or y axis, but I won't talk about those. More information on the `Axes` class [here](https://matplotlib.org/3.2.1/api/axes_api.html#the-axes-class).

#### Current figure/axes
When you have matplotlib.pyplot imported, there is effectively always a current `Figure` and `Axes` object. You can access these through the functions `plt.gcf()` and `plt.gca()` (get current figure/axes). These are what are used whenever you call plot functions directly from `plt` such as `plt.plot(...)`. You can also create your own `Figure` and `Axes` with the functions `plt.figure()` and `plt.subplot()` (these are probably the most commonly used, but there are lots of other ways to make these objects). You can have multiple `Figure` and `Axes` objects created, and edit all of them concurrently, but only one of each will be the "current" one reached through `plt.` methods. Lastly, the current `Figure` _must_ contain the current `Axes`, so if you switch axes objects, the current figure may change automatically.

You can switch between `Axes` objects using `plt.sca(ax)` (set current axes), where `ax` is the `Axes` object to switch to. If you want to switch back to the last one, you'll need to save it to a variable somewhere!

You can switch between `Figure` objects using the `number` attribute. If you have `Figure` object `fig`, you'd call `plt.figure(fig.number)` to switch to `fig`.

In [ ]:
# Current figures/axes example
fig1 = plt.figure()
ax1 = plt.subplot()

fig2 = plt.figure()
ax2 = plt.subplot()
print("We just created fig2.")
print("Current figure is fig2?\n", plt.gcf() is fig2)
print("Current axes is ax2?\n", plt.gca() is ax2)
print()

plt.sca(ax1)
print("We just switched to ax1.")
print("Current axes is ax1 now?\n", plt.gca() is ax1)
print("And we automatically switched to fig1?\n", plt.gcf() is fig1)

plt.figure(fig2.number)
print("We just switched to fig2.")
print("Current figure is fig2?\n", plt.gcf() is fig2)
print("Current axes is ax2\n", plt.gca() is ax2)
print()

try:
    print("To switch figures, can't we just call plt.scf()?")
    plt.scf(fig1)
except Exception as e:
    print("We could try, but", end=" ")
    print(e)

try:
    print("How about just calling plt.figure(fig2)?")
    plt.figure(fig2)
except Exception as e:
    print("Sure, but", end=" ")
    print(e)

fig1.text(0.5, 0.5, "FIGURE 1", horizontalalignment='center', fontsize=18) # same as plt.figtext()
fig2.text(0.5, 0.5, "FIGURE 2", ha='center', fontsize=18); # 0.5, 0.5 is in window coordinates (0 to 1)

Most of the same `plt.` plotting/formatting functions are available as methods of `Axes`, like `plot`, `imshow`, etc. I always just try it first, and if it throws an error, Google it.
Some are named differently but behave the same, like `plt.xlabel` versus `ax.set_xlabel`. See the [source code](https://matplotlib.org/3.1.1/_modules/matplotlib/pyplot.html#xlabel) for `plt.xlabel`; it just calls `plt.gca().set_xlabel()`.
Some things behave slightly differently, like `plt.xlim` and `ax.set_xlim`/`ax.get_xlim`. See the [source code](https://matplotlib.org/3.1.1/_modules/matplotlib/pyplot.html#xlim) for `plt.xlim`; it checks for arguments and decides which `Axes` method to call.

#### Do I have to use `Figure` and `Axes` objects all the time??
No, it's not always necessary to use this object-oriented approach to plotting. It can be useful in situations where you're looping over something and want to make several plots each iteration, or when you want to generalize a plotting function to take an `Axes` object as an argument, so that you don't have to copy-paste as much code! My general rule is "do whatever is fastest and easiest". Minimize time spent both writing and running code, but also time spent scrolling through your .py file or re-reading 10000 lines of plot copypasta from 2 years ago.

#### Into the Weeds
This doesn't really help you plot better, so you can skip this, but it's interesting! If you check the type of the result of a `plt.subplot()` call, the returned object isn't `Axes`, it's `AxesSubplot`. I'm not sure of the specifics here, but it seems like matplotlib gives you a subclass of both `Axes` and `SubplotBase`, which has methods for arranging multiple subplots.

The SubplotBase [documentation page](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.axes.SubplotBase.html#matplotlib.axes.SubplotBase) explains some of what that class does, and the source code for this function shows how they dynamically generated that `AxesSubplot` class! Learn more about dynamic class creation [here](https://www.python-course.eu/python3_classes_and_type.php) and [here](https://www.geeksforgeeks.org/create-classes-dynamically-in-python/).

***
### 3.2 plt.subplot2grid
You might be familiar with placing multiple subplots in one figure, and the associated [plt.subplot()](https://matplotlib.org/3.3.1/api/_as_gen/matplotlib.pyplot.subplot.html) function for making one subplot at a time. To make multiple at a time (and the figure), you can use [plt.subplots](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.subplots.html)&mdash;this one is neat because you can use the `sharex` and `sharey` keywords to tie the axis limits of all the subplots together. But unless you do something really fancy or hacky, these only generate equally-sized subplots on regular grids.

[plt.subplot2grid()](https://matplotlib.org/3.3.1/api/_as_gen/matplotlib.pyplot.subplot2grid.html) is a nice way to arrange subplots of varying sizes and dimensions in one figure. Check out the documentation for all the rules. 

On that documentation page under "Notes", it points out that this function is a wrapper function for Gridspec, which I am not personally familiar with. If you get really into making these kinds of plots, you might check out Gridspec.

In [ ]:
# Make a new figure
# If we were using a window viewer, this text would be the window title
fig1 = plt.figure("fresh figure")

# Define the entire grid
nrows, ncols = 7, 7
# Define the upper left corner of this subplot
row, col = 0, 0
# Define the right-ward and downward lengths of the subplot
rowspan, colspan = 3, 3

# Make the first subplot using the above definitions
ax1 = plt.subplot2grid((nrows, ncols), (row, col), rowspan, colspan)
ax1.fill_between(x=np.arange(10), y1=np.arange(10), y2=np.full(10, 9), color='#008080')

# Make another subplot on the same grid, but in a different location and with different dimensions
ax2 = plt.subplot2grid((nrows, ncols), (0, 4), 7, 4)
ax2.plot(np.sin(np.arange(40)*np.pi/20), np.arange(40)-20)

# Make another subplot defined on a different grid
ax3 = plt.subplot2grid((8, 8), (4, 0), 4, 4)
ax3.fill_between([0, 1, 2, 4, 6, 7, 8], [0, 2, 1.5, 3, 1, 1.5, 0], ec='k', lw=2, alpha=0.4);

***
### 3.3 Other useful plotting functions
I'll run through some examples of other useful plotting functions
#### 3.3.1 `imshow`
Show an image, represented by a 2D array, like one pulled from a standard FITS file. You can apply stretches and limits to improve the displayed image.

In [ ]:
# Get a test file from astropy
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
fits_image_filename = get_pkg_data_filename('tutorials/FITS-images/HorseHead.fits')

img = fits.getdata(fits_image_filename)

plt.figure(figsize=(16, 8))
plt.subplot(121)
# Use imshow. Set origin='lower' to make sure image is oriented correctly
plt.imshow(img, origin='lower')

# We can try to improve the plot by changing the limits and stretch
plt.subplot(122)
# Use quantile-based approach to vlims. Still usually requires trial and error (for me)
img_sorted = np.sort(img.flatten())
vmin, vmax = img_sorted[img_sorted.size//100], img_sorted[99*img_sorted.size//100]
stretch_fn = lambda x: np.arcsinh(x)
plt.imshow(stretch_fn(img), origin='lower', vmin=stretch_fn(vmin), vmax=stretch_fn(vmax))

del img # save memory while we're not working here

You can also show RGB images with imshow. You'll have to stack them so the RGB axis is the last axis, i.e. shape of `[M, N, 3]`.

In [ ]:
imgs = []
for band in ['g', 'i', 'r']:
    fits_image_filename = get_pkg_data_filename(f'visualization/reprojected_sdss_{band}.fits.bz2')
    imgs.append(fits.getdata(fits_image_filename))

# Apply a log stretch
# matplotlib.colors offers LogNorm, but it didn't work here for some reason :/
plt.figure(figsize=(16, 8))
plt.subplot(111)
stretch_fn = np.log
for img in imgs:
    # Ignore negatives
    img_sorted = np.sort(img[img > 0].flatten())
    # I usually need to find good vmin/vmax by trial and error...
    vmin, vmax = img_sorted[2*img_sorted.size//3], img_sorted[399*img_sorted.size//400]
    # Clip the data to the limits and rescale to the (0, 1) interval
    img[img < vmin] = vmin
    img[img > vmax] = vmax
    img[:] = stretch_fn(img)
    img -= np.nanmin(img)
    img /= np.nanmax(img)
# Make (N, M, 3) array for imshow
imgs = np.stack(imgs, axis=-1)
plt.imshow(imgs, origin='lower')
del imgs, img

#### 3.3.2 Patches and Colors
Matplotlib patches are what the software uses for plotting behind the scenes, but it's sometimes convenient to use them directly. `Patch` is a subclass of `Artist`, [which runs a lot of matplotlib objects behind the scenes](https://matplotlib.org/3.3.1/api/artist_api.html), including that `Axis` object I mentioned earlier. Any kind of artist can be added to an `Axes` instance with the method `ax.add_artist`.

There are a [bunch of different kinds](https://matplotlib.org/3.3.1/api/patches_api.html) of Patches. See [the documentation](https://matplotlib.org/3.3.1/api/_as_gen/matplotlib.patches.Patch.html) for the `Patch` class for common methods.

When you've got a lot of colors on one plot, it's nice to have a little more control over the particular shades, instead of working strictly with red, blue, green, etc. Matplotlib accepts names of extended [web colors](https://en.wikipedia.org/wiki/Web_colors), like `Tomato`, `Thistle`, and `MintCream`. If you want to get creative, you can also specify colors by their RGB hex codes with a leading `#` sign, like `#FF6347` (`Tomato`). The first two characters specify red, the second two green, and the last two blue. They're in hexadecimal, which base-16, so for `10` in base-10, hex uses `A`, up to `F` for base-10 `15`. If you [convert to base-10](https://www.binaryhexconverter.com/hex-to-decimal-converter), `FF 63 47` is `255 99 71`. You can also specify colors to Matplotlib as a tuple of 3 floats in the `[0, 1]` interval, so if you have them as integers from `[0, 255]`, you can just divide by 255. Use a color picker like [this one](https://image-color.com/color-picker.html) to visualize the colors you're specifying and generate the RGB values. The one I linked even suggests other colors for a palette. You can google "color picker rgb" or something, there's tons of these.

One final thing to consider when choosing colors is how friendly they are to [colorblind](https://www.nei.nih.gov/learn-about-eye-health/eye-conditions-and-diseases/color-blindness/types-color-blindness) readers. When you're making plots for publication or presentation, consider running them through a [colorblindness simulator like this one](https://www.color-blindness.com/coblis-color-blindness-simulator/) and iterating through color schemes until you find a good one. You can also rely on linestyles (for lines) and hatching (for color/2D) to aid in differentiating different lines or regions. I have heard there are also colormaps already designed to be inclusive in this regard. I haven't read up on this extensively, but a quick search turns up [articles like this](https://www.ascb.org/science-news/how-to-make-scientific-figures-accessible-to-readers-with-color-blindness/) which help guide more inclusive figure-making!

In [ ]:
from matplotlib import patches

plt.figure(figsize=(8, 8))
ax = plt.subplot()

# Circle, and set a couple appearance keywords
circle1 = patches.Circle((0.5, 0.5), 0.23, facecolor='#F4A460', edgecolor='white')
circle2 = patches.Circle((0.5, 0.5), 0.2, facecolor='Moccasin', lw=0)
ax.add_artist(circle1)
ax.add_artist(circle2)
# Add more circles in a loop
rng = np.random.default_rng()
for i in range(15):
    dx, dy = 1., 1.
    while (dx**2 + dy**2) > 0.18**2:
        dx, dy = rng.uniform(low=-0.2, high=0.2, size=2)
    circle3 = patches.Circle((0.5+dx, 0.5+dy), 0.02, facecolor='red', lw=0)
    ax.add_artist(circle3)

# Add a wedge
wedge1 = patches.Wedge((0.5, 0.5), 0.23, 5, 35, color='white')
ax.add_artist(wedge1)
# This wedge was placed afterward the previous patches, so it goes "on top" of them
# You can manually control where in the "stack" of patches this goes using the "zorder" keyword
# If you need your patch to go on top, use a high value of zorder (like 10000), to place it "on top"


# Add an arrow to indicate the other patches
arrow = patches.Arrow(0.1, 0.9, 0.2, -0.2, width=0.2, color=tuple(x/255. for x in (69, 179, 113)))
ax.add_artist(arrow)

# Add text
ax.text(0.5, 0.8, "someday we'll have\npayday pizza again", ha='center', fontsize=13);

***
***
<a id='sec_astropy'></a>
## 4. `astropy`

`astropy` is a core Python package for astronomical computing that implements a variety of useful functions. [You can read more about the project here](https://www.astropy.org/). `astropy` should already be included in your Anaconda distribution of Python, so you shouldn't normally have to worry about downloading or installing it separately.

Just like with `numpy` and `scipy`, you can save yourself tons of time and headache by looking for functions in `astropy` before you try to write them yourself. `astropy` has a surprising number of convenient functions and classes ready to go. Stuck on your [unit](https://docs.astropy.org/en/stable/units/) conversions, or found yourself on that [NRAO page](https://science.nrao.edu/facilities/vla/proposing/TBconv) about converting flux to [brightness temperature](https://docs.astropy.org/en/stable/api/astropy.units.brightness_temperature.html#astropy.units.brightness_temperature)?? `astropy` has what you need.

***
### 4.1 FITS I/O
I don't know about theorists, but most observers will deal in FITS files pretty frequently. To be honest, I don't know how to access their data in Python _without_ `astropy`.

FITS files contain informative headers, which have their own [standard, archaic format](https://fits.gsfc.nasa.gov/fits_dictionary.html), and data, which is either a table or _n_-D array. I'm not as familiar with the table version, but `astropy.io.fits` can handle that just fine too.

`astropy.io.fits` implements several Python classes that represent various parts of these files. The headers are represented by the `Header` class, which behaves like (but is not subclassed from) a Python `dict`. The image data arrays are represented by `ImageHDU` objects ([read more here](https://docs.astropy.org/en/stable/io/fits/api/images.html#imagehdu)). According to [the main astropy FITS page](https://docs.astropy.org/en/stable/io/fits/), "an HDU (Header Data Unit) is the highest level component of the FITS file structure, consisting of a header and (typically) a data array or table". Objects representing HDUs (usually `ImageHDU`) are wrapped in `list`-like `HDUList` objects when loaded from or saved to files.

FITS files sometimes contain several extensions, which are similarly-shaped data arrays with their own headers all wrapped up in one file. For example, the first extension might be the data, the second might be uncertainty, the third might be coverage, etc.
A multi-extension FITS file will sometimes have a global header, one header without any data.
For these, the global header is contained in the `PrimaryHDU` object, and the image extensions are contained in `ImageHDU` objects. All of these are collected in an `HDUList`, with the `PrimaryHDU` first.

In [ ]:
from astropy.io import fits

# This is just for demo purposes, astropy has some nice test images
from astropy.utils.data import get_pkg_data_filename
fits_image_filename = get_pkg_data_filename('tutorials/FITS-images/HorseHead.fits')
# Ordinarily you'd have your own FITS file, since you wouldn't just be doing this for fun

# Get the data from a single-extension file using the getdata convenience function
# If the file is multi-extension, gets the first HDU with data
img, hdr = fits.getdata(fits_image_filename, header=True)
print(f"Loaded an image with dimensions {img.shape}")
print(f"The image was observed at {hdr['TELESCOP']} using {hdr['INSTRUME']}")
# You could print the whole header, too
print_long_thing = False
if print_long_thing:
    print(hdr.tostring(sep='\n'))
    
del img, hdr
# Open the HDUL manually (useful for multi-extension cubes, or when you don't know if it's single or multi)
# The "with" statement helps you open and close files/streams cleanly!
with fits.open(fits_image_filename) as hdul:
    print(f"This file has {len(hdul)} extensions")
    print()
    if len(hdul) > 1:
        for i in range(len(hdul)):
            print(f"The {i}th extension is a {type(hdul[0]).__name__}")
            if 'EXTNAME' in hdul[i].header:
                print(f"The extension is named \"{hdul[i].header['EXTNAME']}\"")
            else:
                print("The extension doesn't have a name")
            if hasattr(hdul[i].data, 'shape'):
                print(f"It contains an array of shape {hdul[i].data.shape}")
            print()

The [intro page](https://docs.astropy.org/en/stable/io/fits/) for this package has thorough instructions on how to open and save FITS files, so I won't spend too much time on this. One thing I have found useful in the past that I'll share is some boilerplate for writing your own FITS header.

Usually when saving FITS files, I just copy a header from one that I opened, but sometimes that doesn't make sense, or I feel like the info in the header would be misleading. In those cases, I'll make a bare-bones header with some useful keywords.

In [ ]:
import datetime 

# Quick example of the header keywords I might throw in (taken from my code)

hdu = fits.PrimaryHDU()
hdu.header['EXTNAME'] = 'T' # this was a temperature
hdu.header['BUNIT'] = 'K'
hdu.header['OBJECT'] = 'perseus'
hdu.header['DATE'] = (datetime.datetime.now(datetime.timezone.utc).astimezone().isoformat(), "File creation date")

try:
    # when your script is in a regular .py file, __file__ will give the filename
    hdu.header['CREATOR'] = (f"Ramsey: {__file__}", "FITS file creator")
except NameError as e:
    print(f"This is a jupyter notebook, so {e}")
    # jupyter notebook doesn't define __file__ :/
    hdu.header['CREATOR'] = ("Ramsey: jupyter notebook", "FITS file creator")
hdu.header['HISTORY'] = "Ramsey wrote this inpainting code on on Jan 13 2020."
hdu.header['HISTORY'] = f"Cutoff: N={1e21:4.0E}. Value from talks with LGM."
hdu.data = np.zeros((20, 20)) # or some more important data
hdu.header['COMMENT'] = "Other important notes you want to put in"
hdulnew = fits.HDUList([hdu]);

# hdulnew.writeto("some_filename.fits")


### 4.2 WCS
I am absolutely _not_ going very deep into WCS (world coordinate system), because I jump down that rabbit hole every 2 months or so, and I've never seen the end. I don't know how old that system is. Too old. Too archaic.
The `astropy.wcs` library makes it pretty easy to work with this stuff without having to know whatever horrors lie underneath. It's all fun and games until your FITS header has some tiny little problem.

One big warning about the (current) `astropy` WCS package: it's heavily based on (or a wrapper for) some other older code, `wcslib`. I don't know how deep this thing goes.

[WCS](https://fits.gsfc.nasa.gov/fits_wcs.html) is how FITS files representing astronomical images or data define the spatial coverage of their data. Specifically, it's a set of FITS header keywords that define a sky coordinate for each pixel in the image. Those keywords generally define a reference pixel somewhere in the image and a reference coordinate (like RA, DEC) to tie to that pixel, and then they define a matrix that transforms pixel distances to sky coordinate distances. If anyone wants to talk more about this, I would love to learn from you about this or share what I've learned.

To get WCS information from a FITS file, you just call the `WCS` object constructor on the FITS `Header`.

In [ ]:
from astropy.wcs import WCS

fits_image_filename = get_pkg_data_filename('tutorials/FITS-images/HorseHead.fits')
img, hdr = fits.getdata(fits_image_filename, header=True)

wcs_object = WCS(hdr)

print(wcs_object)

You can use this WCS object to find the sky coordinates of given pixels (`wcs_object.pixel_to_world`) (although watch out, pixels are `xy`-indexed, which is the opposite of `numpy` array indexing), or the pixel coordinates of a sky coordinate (`wcs_object.world_to_pixel`). Or if the flipped pixel index bothers you, you can use the `array_index_to_world` (and vice versa) versions of those functions&mdash;but watch out, array indices need to be integers!! LOL!!

Another useful function `astropy.wcs` offers is `wcs_obj.footprint_contains`, which will tell you if a given coordinate is contained in this image.

Check out the [list of WCS class attributes and methods](https://docs.astropy.org/en/stable/api/astropy.wcs.WCS.html#astropy.wcs.WCS) to see what functions you might use.

***

### 4.3 SkyCoord
SkyCoord has significantly improved my life. From the `astropy` [coordinates](https://docs.astropy.org/en/stable/coordinates/index.html) library, this class representes astronomical sky coordinates _and does all the transformations between different coordinate systems_. You can read in sky coordinates in a variety of formats and print them out different ways too. You can calculate the angular separation or position angle between coordinates, or find a coordinate given an angular separation and position angle from another. It's great, I love it.

Certain `WCS` functions naturally return `SkyCoord` objects. I sometimes use the following trick to find the pixel scale of an image directly. If you have better suggestions of how to do this, I'd love to learn!

In [ ]:
from astropy.coordinates import SkyCoord

# Grab the coordinates of two adjacent pixels
c1, c2 = wcs_object.pixel_to_world(0, 0), wcs_object.pixel_to_world(0, 1)
pixel_scale = c1.separation(c2)
print(f"The pixel scale of the image is {pixel_scale}")

Pretty cool, right? That pixel scale even has units! Would be nice to convert those degrees into something more reasonable...

***

### 4.4 Units
I only discovered the power of `astropy.units` somewhat recently. [This library](https://docs.astropy.org/en/stable/units/) has `Unit` and `Quantity` classes that represent base units and values with associated units, respectively. You would mostly work with `Quantity` objects, which is what happens when you multiply a number or array by a `Unit` or another `Quantity`. When you add or multiply `Quantity` objects with different units, it will figure out the conversions to make, or make new units, or yell at you (when appropriate).

Check out the introduction page (linked above), where they give a bunch of neat examples of what you can do with this package.

In [ ]:
from astropy import units as u

# Easy conversions, just use Unit objects like u.arcsec (see the documentation for a full list)
print(f"The pixel scale of the image is {pixel_scale.to(u.arcsec):.2f}")
print(f"1000 pixels would be {(1000*pixel_scale).to(u.arcmin):.2f}")

***
***
<a id='sec_classes'></a>
## 5. Classes and Object-oriented Programming

When you either code directly in your interpreter or write a column of mostly left-aligned code, working line by line in the main file, that's generally called [scripting](https://en.wikipedia.org/wiki/Scripting_language). When you group your code into functions and use those as the main components of your code, that's generally called something like [procedural](https://en.wikipedia.org/wiki/Subroutine) or [functional programming](https://en.wikipedia.org/wiki/Functional_programming). When you organize concepts in your code into data structures that have their own attributes (like data, information) and methods (functions), that's called [object-oriented programming](https://en.wikipedia.org/wiki/Object-oriented_programming) (OOP).

Apologies if I misused any of these terms; I think this is mostly accurate, at least in my experience. Most importantly, there's no hierarchy to these programming styles. Different languages use different paradigms, and Python happens to support all of the styles I mentioned above. All of them have advantages and trade-offs. Use whatever style or combination works best for you for a particular task.

Object-oriented is pretty popular in software, and is the fundamental style of languages like Java and C++. There are lots of resources online on how to write classes in Python, like [this one](https://www.geeksforgeeks.org/python-classes-and-objects/). I'll try to stick to short examples and point out where to look for more information.

***
### 5.1 Classes
When you work with classes, you have the _class,_ which is a sort of abstract concept of the class, like the idea of a tree, and then you have an _instance_ of that class, which is a real version of that class, like the tree you have in your backyard. The idea of a "tree" and the physical tree in your backyard are different. Your tree is a tree, but "a tree" is not _your_ tree. There can be lots of other trees, and they have just as much claim to the idea of trees as yours does. In practice, what this means is there can be class attributes and instance attributes. Instance attributes, what you'll work with primarily, are variables attached to the instance. Instances generally have the same attributes, but the values of those attributes may differ. Like, all trees have leaves and branches, but the specific number of branches or leaves differs.

Classes can also inherit from other classes. You can basically "copy" everything from an existing class, and then modify or add to it. The idea of a tree inherits from the idea of a plant; all trees are plants, not all plants are trees. In OOP, this is called subclassing.

In [ ]:
from astropy.modeling import models

# Define a class, use whatever name is most descriptive
class DustModel:
    """
    DustModel class
    Author: Ramsey Karim
    Created: September 13, 2020

    I don't know the style guidelines for class documentation, but couldn't hurt to have some
    general information here, right?
    """
    
    # the __init__ function initializes the instance of the class
    def __init__(self, T, N, beta=2.0):
        """
        DustModel representing an isothermal column of interstellar dust.
        Model only valid in the far-infrared.
        :param T: Quantity, temperature of the dust
        :param N: Quantity, column density of the dust
        :param beta: (optional) float, far-infrared spectral index
        """
        # initialize the instance attributes
        self.T = T
        self.bb = models.BlackBody(temperature=T)
        self.N = N
        self.beta = beta

    # regular class methods always need "self" as the first argument
    # "self" is the instance!
    def radiate(self, nu):
        """
        Model the flux emitted from the dust at the given frequencies or wavelengths
        :param nu: Quantity, frequency equivalent
        """
        nu = nu.to(u.Hz, equivalencies=u.spectral())
        source_flux = self.bb(nu)
        # optical depth (I think)
        tau = (u.M_p * self.N * (0.1 * (nu/(1000*u.GHz))**self.beta * u.cm**2 / u.g)).decompose()
        return source_flux * (1 - np.exp(-tau))

    # Methods surrounded by underscores are usually special Python things
    # They can help your object interact in more "pythonic" ways
    def __str__(self):
        """
        Some short string to describe the object roughly
        """
        return f"DustModel(T={self.T:.1f})"

    def __repr__(self):
        """
        A little more descriptive (at least, that's how I usually use __repr__)
        """
        return f"<DustModel ({self.T:.1f}), ({self.N:.1E}), (beta={self.beta})>"

    # Can define what happens 
    def __call__(self, *args, **kwargs):
        """
        Call the DustModel.radiate function
        """
        return self.radiate(*args, **kwargs)

    
d = DustModel(25*u.K, 1e21/u.cm**2)
print(d)
d

In [ ]:
wavelength = (10**np.arange(1, 3, 0.05)) * u.micron
flux = d(wavelength)
print(f"Flux originally in units of {flux.unit}")
plt.plot(wavelength.to(u.GHz, equivalencies=u.spectral()), flux.to(u.MJy/u.sr), color='k')
plt.xlabel("Frequency (GHz)")
plt.ylabel(f"Flux (MJy/sr)")
plt.xscale('log');

***
### 5.2 Modular programming
_Quick note about this notebook:_ Modular programming isn't well aligned with how Jupyter notebooks operate, so I won't show any examples in this notebook. You can probably try to use some concepts of modular programming with notebooks (maybe, I've never tried), but it's vastly more common for modular programming to be used with plain `.py` files organized into folders.
***

Another useful programming concept is the idea of [modular programming](https://en.wikipedia.org/wiki/Modular_programming). Modular programming is a way to [organize code](https://www.geeksforgeeks.org/modular-approach-in-programming/) into different files, called "modules". If you're familiar with this style, you may have noticed that it seems similar to object-oriented programming! You define classes which represent some kind of object (like a tree) that has attributes (like number of leaves, root pattern) and methods ("grow taller", "drop branch on car"), and can be instanced (the tree in your back yard or outside your office). When you organize your code into modules, you're organizing a bunch of functions and classes that operate on related data or achieve a general goal. It's a library of functions! They can be used for different things, but they're all related.

Modules behave a little like classes in Python in that you can call functions from them similarly to class methods. However, modules aren't instanced. It wouldn't make sense. In our tree example, a related module might be Gardening, which has functions to prune and water trees, maybe implements a Bee class to help pollinate. All of these fall under the umbrella of gardening, but you don't expect to instance Gardening and hold something in your hand and say "This item is a Gardening".

If you've taken a look at the source code for Python packages, you can see modular programming all over the place.
Check out the [Astropy Github](https://github.com/astropy/astropy/tree/master/astropy) to see the way Astropy organizes their code. If you click the `wcs` library (which is just a folder under `astropy/`), you'll find an `__init__.py` file, some `.py` files, and a couple directories with code in them as well. Those directories also have `.py` files and an `__init__.py` file.

Click on the `__init__.py` file under `astropy/wcs`. It's a short file, mostly importing everything from the `wcs.py` file in that same directory, and then importing the `utils.py` module by name.
`utils.py` is a file being treated as a module of its own. You can look at that file and find a bunch of functions, all related by their general goals of supporting the WCS package, and you can import all of them under the name `utils`, and you can access them by dot-notation, like `utils.add_stokes_axis_to_wcs`.

One quick thing about importing: whenever you import a module, what happens (at least what I believe generally happens) is all the code in that file is "run", so that you have those variable, function, and class definitions ready to go in your current file. If you have left-aligned code, that will run too. When you decide to reuse functions from old code, make sure you don't have unecessarily expensive left-aligned code in that old file. Every time you import stuff from that file, you'll run that code. A way around this is the `if __name__ == "__main__":` statement. If you put all your "left-aligned" code under one of these, it will _ony_ run if you specifically ran this file in terminal; it will not run when this file is imported into another.

Back to Astropy. The `wcs.py` is not so much being treated as a module here. You can import all the functions from it directly, with the `*` notation you see in the `__init__.py` file. This means you will no longer be able to tell the difference between functions you wrote in the current file and functions you imported. You can access the functions just fine, same as with `utils`, but you don't need to use `wcs.<function_name>` dot-notation, you just call `<function_name>`. When you read about this style of importing, you're usually warned against using this too much, since it can cause confusion (especially in larger files) as to whether some function was defined here by you, or was imported from somewhere else.

However, you can also use `from <module> import *` to _hide_ where that stuff came from. That's sort of what they're doing here. First, we need to talk about what that `__init__.py` file is doing. It turns out that, if you were to back up into the `astropy/` directory, you can import the _folder_ `wcs` like a module. The entire _directory_ becomes a module! But it needs that `__init__.py` file to _know_ that it's a module, otherwise you'll just get an error. If you just want to be able to chain through the directory to import specific files underneath, you can leave the `__init__.py` file empty! The issue is that, in that case, you have to know what's under that directory. Let's import the `WCS` object from the `wcs.py` file.

In [ ]:
from astropy.wcs.wcs import WCS

WCS

Great! It worked. But it typing `.wcs.` just to jump through that directory felt redundant. What if the directory just knew what we'd want from it?

The `__init__.py` file can do that too! Like I said earlier, when you import a file, all the code in that file is run. When you import a directory, only what is in the `__init__.py` file is run. If you import something to that `__init__.py` file, then the directory knows about it, and you can just get it from the directory next time!

Since the `wcs/` directory imported `*` from `wcs.py`, it knows about the `WCS` object.

In [ ]:
from astropy.wcs import WCS

WCS

It's the same object! We can make sure:

In [ ]:
from astropy.wcs.wcs import WCS as WCS_fromfile
from astropy.wcs import WCS as WCS_fromdir

print(WCS_fromfile is WCS_fromdir) # They're the same!

We also have access to that `utils` file in the WCS directory. I don't know what the difference between importing that into `__init__.py` or just letting people import that file through the directory is. Maybe it's more clear that this is part of the API? Either way, it's the exact same import statement:

In [ ]:
from astropy.wcs import utils
# or something like this, since "utils" is pretty nonspecific
from astropy import wcs
# It's the same library at the end of the day
print(wcs.utils is utils)

This style of programming works great with the concept of [abstraction barriers](https://en.wikibooks.org/wiki/Object_Oriented_Programming/Abstraction_Barrier), which I will not go into here, but which I recommend reading about if you're interested in designing software!

Modular programming is something I am actively trying to learn more about! I think it's a really neat way to organize code, especially if I plan to put stuff on Github for others to see/use. If anyone has experience with this, I'd love to learn more!

***
<a id='sec_builtins'></a>
## 6. Python built-ins
The Python language has a [small number of built-in functions/classes](https://docs.python.org/3/library/functions.html). These are the core of the language, and even though we rely heavily on `numpy`, `scipy`, `astropy`, and other external libraries, these built-ins are still powerful and can be used effectively!

A few of these are not functions, but objects, and you're probably very familiar with some of them. There's primarily the `list`, the dictionariy `dict`, the `tuple`, and the `set` and `frozenset`.

***
### 6.1 Mutable vs immutable types
Objects in Python can either be mutable or immutable. [This article](https://medium.com/@meghamohan/mutable-and-immutable-side-of-python-c2145cf72747) gives a thorough explanation of these concepts (with a cool figure at the beginning!), but in brief, mutable objects can have their contents changed, and immutable objects cannot.
A `list` is mutable, but a `tuple` is not. Check this out:

In [ ]:
print("list:")
a = [10, 20]
print(f"a[0] = {a[0]}")
a[0] = 30
print("a:", a)
print()

print("tuple")
b = (10, 20)
print(f"b[0] = {b[0]}")
try:
    b[0] = 30
except Exception as e:
    print(f"There was a problem: {e}")
print("b:", b)

We are unable to modify the tuple, since it is immutable. This isn't necessarily a bad thing; it can be useful to know that certain values cannot be changed!

***
### 6.2 Hashing
Another concept worth mentioning here is [hashing](https://en.wikipedia.org/wiki/Hash_function), which the generalized act of converting an arbitrary chunk of data (like any object in Python) into a fixed-length value (like an integer). The name pretty much means "to chop up" or "make a mess of", like a breakfast hash! One of the key qualities of a hash function is that it should map a unique object to a unique number, as best as possible. If two instances of objects can be considered "equal", then they should has to the same number, and if they can be considered "unequal", they should hash to different numbers. These are used for a variety of computer science data structures and algorithms, including cryptography, but more relevant to us, the concept of [hash tables](https://en.wikipedia.org/wiki/Hash_table).

#### 6.2.1 Motivation for hash tables
You can definitely read the Wikipedia articles and search for other resources on this if you're interested, but I'll give a brief motivation for why this stuff is useful to us.

When you make a list with a number of elements, and you want to retrieve and element, you can do so either by the index or by the item itself. Since a Python `list` is implemented with a C-array, accessing by index is really fast! Your computer knows exactly where to find that element, since you effectively gave it the memory address.
But what if you want to find a particular element, and don't know where in the list it might be? In that case, since the C-array and Python `list` don't "know" anything about what they contain, you'd have to search through the entire list until you found what you're looking for.

That means that the longer your list is, the more time this takes. What if you could sort of tell your list what was in it? Picture this: you quickly turn your object into an integer (somehow, doesn't matter how), and you use part of that integer as an index to place your item somewhere in the list. If you want to look for that item in your list, you hash it again and check that area in your list. The tradeoff is that you don't know how your list is actually organized (since the hashing should be done behind the scenes), so you can't place any meaning in the order of the list.

This kind of hash table is implemented in Python as the `set`. You can add items to the `set`, but items in the set need to be unique, and you can't access them by index. It's a nice way of collecting unique items and quickly checking if you already have them.

In [ ]:
a = set()
a.add(10)
a.add(30)
a.add((1, 20))
print(10 in a)
print(a)

Sets have restrictions on what can be put in them. Because hashing should be unique to the object's "value", it must depend on the attributes of that instance (and not the memory location or something like that). Because you need to use the hash function to retrieve the object, that hash function can't change over time. Thus, you can only include in a `set` objects whose values cannot be changed: immutable objects!

In [ ]:
a = set()
l = [10]
t = (10,)
print(f"I have a {type(l).__name__} = {l} and a {type(t).__name__} = {t}")

try:
    a.add(t)
    print("I added the tuple to the set")
except Exception as e:
    print("There was a problem adding the tuple to the set:", e)

try:
    a.add(l)
    print("I added the list to the set")
except Exception as e:
    print("There was a problem adding the list to the set:", e)

Tuples are useful because they can be added to sets! You might store a coordinate pair, something where order matters and the values shouldn't change, to a set to keep track of unique coordinates.

Sets themselves are mutable (you can `.add` to them), so they cannot be added to other sets. The `frozenset` is an immutable hash table, so it can be added to a `set`! You can call the `frozenset` constructor on a sequence (including an existing `set`) to create a new `frozenset` instance. You can use all the same lookup functions, but you cannot add or remove items.

In [ ]:
a = set()
a.add(40)
a.add(50)
b = set()
b.add((1, 20))
print(f"set b = {b}")

# Try to add a to set b
try:
    b.add(a)
except Exception as e:
    print("There was a problem adding set a to set b:", e)

frozen_a = frozenset(a)
try:
    b.add(frozen_a)
    print(f"Added frozenset(a): set b = {b}")
except Exception as e:
    print("There was a problem (but there shouldn't be!)")

#### 6.2.2 Frozenset example

The `frozenset` class can be useful for representing sequences of objects whose order shouldn't be considered, like the vertices of a triangle. To represent a triangle as a properly hashable construct, you could represent your coordinates as tuples (order matters, hashable), and then store 3 tuples in a `frozenset`, where the order doesn't matter, so any combination of those three coordinates will return an "identical" object.

In [ ]:
class Triangle:
    """
    Triangle class
    Author: Ramsey Karim
    Created: September 13, 2020
    """
    def __init__(self, *coords):
        """
        Triangle represents a 3-pointed triangle in n-D
        :param coords: sequence arguments representing the vertices. All sequences must be the same length.
            Exactly 3 sequences must be given as arguments.
        """

        # Make sure you gave 3 arguments
        try:
            assert len(coords) == 3
        except AssertionError:
            raise RuntimeError(f"You cannot make a triangle out of {len(coords)} vertices")
        
        # Make sure your arguments are sequences
        try:
            assert (hasattr(c, '__iter__') for c in coords)
        except AssertionError:
            raise RuntimeError("Your coordinates must be sequences")
        
        # Convert coordinates to tuples (just in case)
        coords_as_tuples = [tuple(c) for c in coords]
        # Make sure all tuples are the same length
        self.n_dim = len(coords_as_tuples[0])
        try:
            assert all(len(c) == self.n_dim for c in coords_as_tuples)
        except:
            raise RuntimeError("Your coordinates must be of the same dimension")
        
        self.vertices = frozenset(coords_as_tuples)
    
    def __str__(self):
        return f"Triangle{str(self.vertices).replace('frozenset', '')}"
    
    def __repr__(self):
        return str(self)

    def __hash__(self):
        """
        Hash on the internal frozenset(tuple) structure
        """
        return hash(self.vertices)

    def __eq__(self, other):
        """
        Rely on the frozenset(tuple) for equality
        :param other: the object being compared to (should be a Triangle)
        """
        return self.vertices == other.vertices


# Now let's test this structure!
first_coords = [(0, 0), (0, 2), (1, 1)]
second_coords = [(0, 0), (0, 1), (1, 0)]
same_coords_as_first_coords = [(0, 2), (1, 1), (0, 0)]

first_tri = Triangle(*first_coords)
same_as_first_tri = Triangle(*first_coords) # same coordinate tuple objects
second_tri = Triangle(*second_coords)
same_coords_as_first_tri = Triangle(*same_coords_as_first_coords) # same numbers but different tuple objects

print(first_tri)
print(f"Same as itself? {first_tri == first_tri}")
print(f"Same as a Triangle with same coordinate tuple objects? {first_tri == same_as_first_tri}")
print(f"Same as a Triangle with same coords, different tuple objects? {first_tri == same_coords_as_first_tri}")
print(f"Same as a DIFFERENT triangle? {first_tri == second_tri}")
print()

a = set()
a.add(first_tri)
a.add(second_tri)
print(f"Set a = {a}")

print(f"Can we find identical Triangles (but different Triangle objects) in the set?")
print(same_as_first_tri in a)
print(same_coords_as_first_tri in a)
print()

print(f"Are they the same objects?")
print(same_as_first_tri is a)
print(same_coords_as_first_tri is a)

Finally, we have the `dict` object. You might be familiar with these as ways to match keys with values, which can be really helpful! It turns out that Python dictionaries are just souped up sets; they're hash maps. They keys are hashed, and the values are looked up with those keys.

Use lists and tuples when order matters, or when you're focusing on the index. Use sets, frozensets, and dictionaries when you're focusing on unique membership, and order doesn't matter. And remember that, for small lists (10s of elements), the performance really doesn't matter.

***
### 6.3 Generators and other iterables
All the types we just went through represent some sort of sequence of objects. The dictionaries are a little special, since they have both the keys and the values, but you could consider a sequence of key-value pairs (in concept). All these "sequence" types are __iterable__, which conceptually means they represent some sequence of objects, and more specifically means (in Python) that they have the `__iter__` method.

When the `__iter__` attribute is called, an object called an `iterator` is returned. An `iterator` is a single-use object that has a `__next__` method. Each time you call `__next__`, the `iterator` returns an object from the sequence. When it runs out of objects to return, it throws an error to say that it's empty. At that point, you just throw it out, you don't reuse it.

For-loops in Python call the `__iter__` method of an iterable object and then loop over the returned `iterator` object. You can call this yourself with the built-in `iter` function, and you can call the `__next__` method with the built-in `next` function.

In [ ]:
a = [1, 2, 3, 4]
print(f"List a is iterable? {hasattr(a, '__iter__')}")

iter_a = iter(a)
print(f"Iter(a) = {iter_a}")

print(f"next(a) = {next(iter_a)}")
print(f"next(a) = {next(iter_a)}")
print(f"next(a) = {next(iter_a)}")
print(f"next(a) = {next(iter_a)}")
try:
    print(f"next(a) = {next(iter_a)}")
except Exception as e:
    print("Done! ", type(e).__name__)

print()
print(f"List a = {a} is unchanged!")

You can implement the `__iter__` method of your own class if you want to be able to naturally loop over it; you'd probably just want to return the `iterator` from some iterable attribute. If you decide to try this, look into it further! I have only tried this once and did not do extensive testing, so I am definitely no expert. I just think it's super cool :-)

You could also build your own iterable from scratch, in a way, using a `generator`. That's another Python type, but you can define them a little differently than most of these types. Check this out.

In [ ]:
# Define generator function. This function, when called, returns a generator object, which works like an iterator
def fibonacci():
    n_minus_1 = 0
    n = 1
    while True:
        yield n + n_minus_1
        n_plus_1 = n + n_minus_1
        n_minus_1 = n
        n = n_plus_1

print("The first two fibonacci numbers are 0 and 1")
fib_iter = fibonacci()
for i, f in enumerate(fib_iter):
    if i > 15:
        break
    print(f"The {i+3:2d}th fibonacci number is {f:5d}")

print()

# Start over with new iterator
fib_iter = fibonacci()
i, f = 0, 0
print("The first two fibonacci numbers are 0 and 1")
while i < 16:
    i += 1
    f = next(fib_iter)
    print(f"The {i+3:2d}th fibonacci number is {f:5d}")

print()
print(type(fibonacci))
print(type(fib_iter))

This is pretty cool, because I'm able to loop over the Fibonacci sequence like it's all in some single list. I don't even know it's not, right? But it can't possibly be in a single list, because it's infinite. One of the powers of generators is [lazy evaluation](https://en.wikipedia.org/wiki/Lazy_evaluation), which means that the items in this kind of sequence aren't created until they're needed. This can allow you to iterate over infinite sequences (and make your own stop conditions in other parts of the code), or to iterate over a sequence of very memory-intensive objects without putting them all in a list at once.

Finally, there is a really nice shorthand for creating all these sequence types: [comprehensions](https://www.geeksforgeeks.org/comprehensions-in-python/).

In [ ]:
# List comprehension
x2 = [x**2 for x in range(10)]

# Can also filter
x_even = [x for x in range(20) if x%2 == 0]

# Tuple comprehension
t = tuple(x**3 for x in x2)

# Set comprehension
s = {x + 3 for x in t}

# Dict comprehension
d = {x: y for x, y in zip(x2, x_even)}

# Generator comprehension
g = (np.full((20, 20), x) for x in s)

print("List", x2)
print("List2", x_even)
print("Tuple", t)
print("Set", s)
print("Dict", d)
print("Generator", g) # Hasn't evaluated anything yet

***
***
## Conclusion
That's all for now! Please let me know if you have questions or want to know more about any of this. I'd love to look into it with you, I really enjoy thinking and talking about this stuff. Happy coding!